In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from os import environ

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
environ["HIP_VISIBLE_DEVICES"]="0"

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    count = torch.cuda.device_count()

__CUDNN VERSION: 3001000
__Number CUDA Devices: 1


In [3]:
model_name = "microsoft/Phi-3-mini-128k-instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    device_map="cuda",
    trust_remote_code=True
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.60s/it]


In [6]:
system_prompt = "You are a programmer that takes in sparse descriptions of a psychology experiment from a scientific paper and generates a PsychoPy Psyexp experiment file that can execute the described experiment."
query = "In this task participants are required to memorise and recall number series in order. Participants start out with three 3-digit sequences. If participants correctly recall 2 out of 3 three sequences, they progress to 4-digit sequence trials and so on. If participants respond incorrectly on 2/3 trials the experiment terminantes. This experiment is based on the original digit span experiment by Jacobs (1887)."

In [10]:
messages = [{"role": "user", "content": f"{system_prompt}\n Input: {query}"}]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [11]:
outputs = model.generate(inputs, max_new_tokens=100_000)
tokenizer.batch_decode(outputs)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


'<s><|user|> You are a programmer that takes in sparse descriptions of a psychology experiment from a scientific paper and generates a PsychoPy Psyexp experiment file that can execute the described experiment.\n Input: In this task participants are required to memorise and recall number series in order. Participants start out with three 3-digit sequences. If participants correctly recall 2 out of 3 three sequences, they progress to 4-digit sequence trials and so on. If participants respond incorrectly on 2/3 trials the experiment terminantes. This experiment is based on the original digit span experiment by Jacobs (1887).<|end|><|assistant|> ```python\n\nfrom psychopy import visual, core, event\n\nfrom psychopy.tools.attributestack import AttributeStack\n\nfrom psychopy.tools.psytoolstub import PsyToolStub\n\n\nclass DigitSpanExperiment(PsyToolStub):\n\n    def __init__(self, *args, **kwargs):\n\n        super().__init__(*args, **kwargs)\n\n        self.sequence_lengths = [3, 4, 5, 6] 